#  BM20A6100 Advanced Data Analysis and Machine Learning
## Erik Kuitunen, 0537275

Import packages

In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import nltk

from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras.models import Sequential

from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

2024-11-18 15:13:24.570446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731935604.731132   14862 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731935604.771270   14862 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 15:13:25.055650: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Read and preprocess text data

In [ ]:
file = open("robinhood.txt", 'rb')
lines = []
for line in file:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    
    if len(line) == 0:
        continue
    lines.append(line)
    
file.close()
text = " ".join(lines)

# set of characters that occur in the text
chars = set( [c for c in text] )

# Total items in our vocabulary
unique_chars = len( chars )

# lookup tables to deal with indexes of characters rather than the characters themselves.
char2index = dict( (c, i) for i, c in enumerate( chars ) )
index2char = dict( (i, c) for i, c in enumerate( chars ) )

Reshaping and one-hot encoding of the data

In [3]:
sequence_length = 10
step = 1
input_chars = []
label_chars = []
for i in range( 0, len(text) - sequence_length, step ):
    input_chars.append(text[i:i + sequence_length])
    label_chars.append(text[i + sequence_length])
    
X = np.zeros((len(input_chars), sequence_length, unique_chars), dtype=np.bool)
y = np.zeros((len(input_chars), unique_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
        

Model definition

In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 50

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, 
                    input_shape=(sequence_length, unique_chars), 
                    unroll=True))
model.add(Dense(unique_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

2024-11-18 15:13:30.968620: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/eki/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training the model

In [5]:
for iteration in range(NUM_ITERATIONS):
    print("Iteration #: %d" % (iteration))
    
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

Iteration #: 0


2024-11-18 15:13:32.436731: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 219621640 exceeds 10% of free system memory.


3991/3991 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 2.1877
Iteration #: 1
  12/3991 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 1.8089  

2024-11-18 15:13:59.267147: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 219621640 exceeds 10% of free system memory.


3991/3991 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 1.7423
Iteration #: 2
  21/3991 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 1.5686

...

3991/3991 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 1.5984
Iteration #: 3
  10/3991 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - loss: 1.5815  

2024-11-18 15:14:55.764187: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 219621640 exceeds 10% of free system memory.


3991/3991 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - loss: 1.5239
Iteration #: 4
  24/3991 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 1.4510

2024-11-18 15:15:18.266627: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 219621640 exceeds 10% of free system memory.


3991/3991 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.4727
Iteration #: 5
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - loss: 1.4387
Iteration #: 6
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - loss: 1.4162
Iteration #: 7
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 1.3951
Iteration #: 8
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - loss: 1.3801
Iteration #: 9
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 1.3693
Iteration #: 10
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 1.3553
Iteration #: 11
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 1.3496
Iteration #: 12
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - loss: 1.3383
Iteration #: 13
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - loss: 1.3325
Iteration #: 14
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - loss: 1.3272
Iteration #: 15
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - loss: 1.3231
Iteration #: 16
3991/3991 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 1.3190
Iteration #: 17
3991/3991 ━━━━━━━━━━━━━━━━━━━━

Testing

In [21]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]
predicted_text = test_chars

print("\nGenerating from seed: %s" % (test_chars))
# print(test_chars, end="")

for i in range(70):
    Xtest = np.zeros( (1, sequence_length, unique_chars) )
    for i, ch in enumerate(test_chars):
        Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    predicted_text += ypred
    # print(ypred, end="\n")
    # move forward with test_chars + ypred
    test_chars = test_chars[1:] + ypred


Generating from seed: the other;


In [22]:
predicted_text

'the other; the children was a tree with the sparrow was all the spot in the spin'